Publisher/Subscriber Service
==================

Big Idea:
> Users make posts. Followers subscribe to the posts they are interested in. Newer posts are more relevant. Display posts by a user, posts for a user. Display those followed by a user. Store the user account information with hashed passwords.

In [1]:
!cat pubsub.py

'Simple message publisher/subscriber service'

from collections import namedtuple, deque, defaultdict
import time

Post = namedtuple('Post', ['timestamp', 'user', 'text'])

# post from newest to oldest, inserting or deleting in front of list use deque
posts = deque()
# defaultdict() with deque() simplifies per-user accumulations of posts
user_posts = defaultdict(deque)

def post_message(user, text, timestamp=None):
    timestamp = timestamp or time.time()
    post = Post(timestamp, user, text)
    # deque.appendleft(datum) beats list.insert(0, datum)
    posts.appendleft(post)
    user_posts[user].appendleft(post)


In [2]:
!cat session.py

'Sample data to test the pubsub internals'

from pubsub import *
from pprint import pprint

post_message('guido', 'i love #python type hinting')
post_message('raymondh', '#python tip: use named tuples')
post_message('barry', 'join a band today')
post_message('raymondh', '#python tip: develop interactively')
post_message('barry', 'learn emacs')
post_message('davin', 'teaching #python today')
post_message('raymondh', '#python tip: have fun programming')
post_message('davin', '#camping tip: always take water')
post_message('barry', 'enums rocks')
post_message('raymondh', '#python tip: never mutate while iterating')
post_message('davin', 'coriander and cilantro come from the same plant')

if __name__ == '__main__':
    pprint(posts)


In [4]:
%run -i session.py

deque([Post(timestamp=1568948372.688842, user='davin', text='coriander and cilantro come from the same plant'),
       Post(timestamp=1568948372.6888409, user='raymondh', text='#python tip: never mutate while iterating'),
       Post(timestamp=1568948372.6888397, user='barry', text='enums rocks'),
       Post(timestamp=1568948372.6888387, user='davin', text='#camping tip: always take water'),
       Post(timestamp=1568948372.6888373, user='raymondh', text='#python tip: have fun programming'),
       Post(timestamp=1568948372.6888356, user='davin', text='teaching #python today'),
       Post(timestamp=1568948372.688834, user='barry', text='learn emacs'),
       Post(timestamp=1568948372.6888328, user='raymondh', text='#python tip: develop interactively'),
       Post(timestamp=1568948372.68883, user='barry', text='join a band today'),
       Post(timestamp=1568948372.6888278, user='raymondh', text='#python tip: use named tuples'),
       Post(timestamp=1568948372.6888204, user='guido', 

In [11]:
# print only raymondh posts

In [12]:
!cat session.py

'Sample data to test the pubsub internals'

from pubsub import *
from pprint import pprint

post_message('guido', 'i love #python type hinting')
post_message('raymondh', '#python tip: use named tuples')
post_message('barry', 'join a band today')
post_message('raymondh', '#python tip: develop interactively')
post_message('barry', 'learn emacs')
post_message('davin', 'teaching #python today')
post_message('raymondh', '#python tip: have fun programming')
post_message('davin', '#camping tip: always take water')
post_message('barry', 'enums rocks')
post_message('raymondh', '#python tip: never mutate while iterating')
post_message('davin', 'coriander and cilantro come from the same plant')

if __name__ == '__main__':
    pprint(user_posts['raymondh'])


In [1]:
%run -i session.py

deque([Post(timestamp=1568948582.1635234, user='raymondh', text='#python tip: never mutate while iterating'),
       Post(timestamp=1568948582.163511, user='raymondh', text='#python tip: have fun programming'),
       Post(timestamp=1568948582.1634967, user='raymondh', text='#python tip: develop interactively'),
       Post(timestamp=1568948582.1634843, user='raymondh', text='#python tip: use named tuples')])


In [2]:
# add typing annotations

In [1]:
!cat pubsub.py

'Simple message publisher/subscriber service'

from typing import NamedTuple, DefaultDict
from collections import namedtuple, deque, defaultdict
import time

User = str
Post = NamedTuple('Post', [('timestamp', float), ('user', str) , ('text', str)])

# post from newest to oldest, inserting or deleting in front of list use deque
posts = deque()                     # type: Deque[Post]     # Posts from newest to oldest
# defaultdict() with deque() simplifies per-user accumulations of posts
user_posts = defaultdict(deque)     # type: DefaultDict[User, deque]

def post_message(user: User, text: str, timestamp: float=None) -> None:
    timestamp = timestamp or time.time()
    post = Post(timestamp, user, text)
    # deque.appendleft(datum) beats list.insert(0, datum)
    posts.appendleft(post)
    user_posts[user].appendleft(post)


In [2]:
%run -i session.py

deque([Post(timestamp=1568949266.6886568, user='raymondh', text='#python tip: never mutate while iterating'),
       Post(timestamp=1568949266.6886494, user='raymondh', text='#python tip: have fun programming'),
       Post(timestamp=1568949266.6886413, user='raymondh', text='#python tip: develop interactively'),
       Post(timestamp=1568949266.6886318, user='raymondh', text='#python tip: use named tuples')])


In [3]:
# add followers

In [4]:
!cat pubsub.py

'Simple message publisher/subscriber service'

from typing import NamedTuple, DefaultDict, Deque, Dict, Set
from collections import namedtuple, deque, defaultdict
import time

User = str
Post = NamedTuple('Post', [('timestamp', float), ('user', str) , ('text', str)])

posts = deque()                     # type: Deque[Post]     # Posts from newest to oldest
user_posts = defaultdict(deque)     # type: DefaultDict[User, deque]
following = defaultdict(set)        # type: DefaultDict[User, Set[User]]
followers = defaultdict(set)        # type: DefaultDict[User, Set[User]]

def post_message(user: User, text: str, timestamp: float=None) -> None:
    timestamp = timestamp or time.time()
    post = Post(timestamp, user, text)
    posts.appendleft(post)
    user_posts[user].appendleft(post)

def follow(user: User, followed_user: User) -> None:
    following[user].add(followed_user)
    followers[followed_user].add(user)


In [5]:
!cat session.py

'Sample data to test the pubsub internals'

from pubsub import *
from pprint import pprint

post_message('guido', 'i love #python type hinting')
post_message('raymondh', '#python tip: use named tuples')
post_message('barry', 'join a band today')
post_message('raymondh', '#python tip: develop interactively')
post_message('barry', 'learn emacs')
post_message('davin', 'teaching #python today')
post_message('raymondh', '#python tip: have fun programming')
post_message('davin', '#camping tip: always take water')
post_message('barry', 'enums rocks')
post_message('raymondh', '#python tip: never mutate while iterating')
post_message('davin', 'coriander and cilantro come from the same plant')

follow('davin', followed_user='raymondh')
follow('davin', followed_user='barry')

if __name__ == '__main__':
    pprint(following)
    pprint(followers)


In [1]:
%run -i session.py

defaultdict(<class 'set'>, {'davin': {'barry', 'raymondh'}})
defaultdict(<class 'set'>, {'raymondh': {'davin'}, 'barry': {'davin'}})
